In [1]:
import os
import pandas as pd

In [ ]:
data_type = ['pressure', 'grass', 'solar']

for data_name in data_type:
    path = '../data/raw/' + data_name
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    
    dfs = []
    for f in files:
        dfs.append(pd.read_csv(path + '/' + f))

    df = pd.concat(dfs, ignore_index=True)
    
    df['Datetime'] = pd.to_datetime(df['Date time'], format='%Y%m%d%H%M')
    df = df.drop(columns=['Date time'])
    df = df[df.Datetime.dt.year == 2024]
    df['Datetime'] = df['Datetime'].dt.round('h')
    
    # Grouping for mean aggregation (non-wind case)
    df = df.groupby(['Datetime', 'Automatic Weather Station'], as_index=False).mean()

    # Interpolation across time per station
    df = df.groupby('Automatic Weather Station', as_index=False).apply(
        lambda group: group.sort_values('Datetime').interpolate(method='linear', limit_direction='both')
    ).reset_index(drop=True)

    # Deduplicate again in case of interpolation artifacts
    df = df.groupby(['Datetime', 'Automatic Weather Station'], as_index=False).mean()

    df.to_csv(f'../data/{data_name}.csv', index=False)

/tmp/ipykernel_563636/876501661.py:22: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  lambda group: group.sort_values('Datetime').interpolate(method='linear', limit_direction='both')
/tmp/ipykernel_563636/876501661.py:22: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  lambda group: group.sort_values('Datetime').interpolate(method='linear', limit_direction='both')
/tmp/ipykernel_563636/876501661.py:22: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  lambda group: group.sort_values('Datetime').interpolate(method='linear', limit_direction='both')
/tmp/ipykernel_563636/876501661.py:22: FutureWarning: DataFrame.interpolate 

In [12]:
data_type = ['wind']

for data_name in data_type:
    path = '../data/raw/' + data_name
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    
    dfs = []
    for f in files:
        dfs.append(pd.read_csv(os.path.join(path, f)))

    df = pd.concat(dfs, ignore_index=True)

    # Clean up speed column
    df['10-Minute Mean Speed(km/hour)'] = df['10-Minute Mean Speed(km/hour)'].astype(str)
    df['10-Minute Mean Speed(km/hour)'] = df['10-Minute Mean Speed(km/hour)'].str.replace(r'N/A', '', regex=True)

    # Ensure correct types
    df = df.astype(dtype={
        'Date time': str,
        'Automatic Weather Station': str,
        '10-Minute Mean Wind Direction(Compass points)': str,
        '10-Minute Mean Speed(km/hour)': float,
        '10-Minute Maximum Gust(km/hour)': float
    })

    # Parse and round datetime
    df['Datetime'] = pd.to_datetime(df['Date time'], format='%Y%m%d%H%M')
    df['Datetime'] = df['Datetime'].dt.round('h')

    # Filter by year
    df = df[df.Datetime.dt.year == 2024]

    # Group by rounded Datetime and Station to avoid duplicates
    df = df.groupby(['Datetime', 'Automatic Weather Station'], as_index=False).agg({
        '10-Minute Mean Wind Direction(Compass points)': lambda x: x.mode().iloc[0] if not x.mode().empty else '',
        '10-Minute Mean Speed(km/hour)': 'mean',
        '10-Minute Maximum Gust(km/hour)': 'mean'
    })

    # Interpolation per station
    df = df.sort_values(['Automatic Weather Station', 'Datetime'])
    df = df.groupby('Automatic Weather Station', as_index=False).apply(
        lambda g: g.set_index('Datetime').interpolate(method='linear', limit_direction='both').reset_index()
    ).reset_index(drop=True)
    
    # Save output
    df.to_csv(f'../data/{data_name}.csv', index=False)


/tmp/ipykernel_563636/1883462998.py:9: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(os.path.join(path, f)))
/tmp/ipykernel_563636/1883462998.py:9: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dfs.append(pd.read_csv(os.path.join(path, f)))
/tmp/ipykernel_563636/1883462998.py:43: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  lambda g: g.set_index('Datetime').interpolate(method='linear', limit_direction='both').reset_index()
/tmp/ipykernel_563636/1883462998.py:43: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  lambda g: g.set_index('Datetime').interpolate(method='linear', limit_direction='both').reset_index(